In [23]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from bertviz import model_view

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/gpt2/248dfc3911869ec493c76e65bf2fcf7f615828b0254c12b473182f0f81d3a707?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1714476205&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDQ3NjIwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9ncHQyLzI0OGRmYzM5MTE4NjllYzQ5M2M3NmU2NWJmMmZjZjdmNjE1ODI4YjAyNTRjMTJiNDczMTgyZjBmODFkM2E3MDc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=QJSFh%7ElaE7CMCB4R9mIFQBPdxVfAo8YVBcllK5v4jgMJU6pf1ViU2EbD9GJNmprlTyBkCiF-ayrfqe2diw8v0z6PGjBXzFeeTJrvo4WeQsSPq7sDefI8DXnvocDw0mNOcdRQoBiN8GcGJ9iwgC7XtPCZ1f0JOm6300I3RewuO4fjBSZ3rrvMwR6SSZfbJ9cFgzgar0vswTx2c4Idq63N42HGHhZi%7E5iBWMBFt6RLeRmAIi2hXUFsWI9y1eNO0ZqRgTYUFABxQUkbxrchRfl6e%7Ee6ODdNOgR4DGFMxA4-k5spQT8aTqQg%7Em9K5ccL%7EN2rYdMzQI7Yzm1Cnzkk6yHh2Q__&Key-Pair-Id=KVTP0A1DKRTAX: HTTPSConnectionPool(host='cd

model.safetensors:  54%|#####3    | 294M/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
phrase = 'My friend was right about this class, It is so fun!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, 
                 output_attentions = True, 
                 output_hidden_states = True)

In [12]:
len(response.attentions)

12

In [13]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1398,   11,  632,  318,  523, 1257,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
response.attentions[-1].shape

torch.Size([1, 12, 13, 13])

In [16]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [17]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])

In [18]:
tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 ',',
 'ĠIt',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [21]:
# Layer index 9, head 0, check almost 60% attention
arr = response.attentions[9][0][0]

n_digits = 3

# Create dataframe dari matrix attention
attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach().numpy())

# label col and row
attention_df.columns = tokens
attention_df.index = tokens

attention_df

,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,",",ĠIt,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
",",0.712,0.015,0.007,0.008,0.007,0.018,0.225,0.009,0.000,0.000,0.000,0.000,0.000
ĠIt,0.517,0.005,0.007,0.007,0.012,0.022,0.386,0.027,0.016,0.000,0.000,0.000,0.000
Ġis,0.397,0.006,0.003,0.003,0.004,0.024,0.482,0.015,0.038,0.028,0.000,0.000,0.000


In [24]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [25]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [26]:
response.logits.shape

torch.Size([1, 13, 50257])

In [27]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns = ['sequence up until', 'next token with highest probability']
)

,sequence up until,next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,",",Ġbut
8,ĠIt,'s
9,Ġis,Ġa
